# Tutorial 0: create your own Parameterfile
In this tutorial, you will learn how to create a parameterfile which contains all configurations to successfully delens your data.

To convert this tutorial into a parameterfile, execute `jupyter nbconvert --to script 'Tut0.ipynb'` in your shell.

## Basic imports

In [1]:
import os
from os.path import join as opj
import numpy as np
import healpy as hp

import plancklens
from plancklens import utils
from plancklens import qresp
from plancklens import qest, qecl
from plancklens.qcinv import cd_solve

from plancklens.sims import maps, phas, planck2018_sims
from plancklens.filt import filt_simple, filt_util

from lenscarf import remapping
from lenscarf import utils_scarf, utils_sims
from lenscarf.iterators import cs_iterator as scarf_iterator, steps

from lenscarf.utils import cli
from lenscarf.utils_hp import gauss_beam, almxfl, alm_copy
from lenscarf.opfilt.opfilt_iso_ee_wl import alm_filter_nlev_wl

## Overview
Delensing consists of the following steps which need to be configured,
 * output path
 * general
 * geometry
 * fiducial spectra
 * transfer function
 * quadratic estimator
 * chain descriptor
 * libdir iterator
 * data / simulations

## output path
Let us store the `Dlensalot` output @ `$SCRATCH/Dlensalot/tutorial0/cmbs4_idealized/`

In [4]:
TEMP =  opj(os.environ['SCRATCH'], 'Dlensalot/tutorial0', 'cmbs4_idealized')

# general
For this tutorial, we assume that data goes to ell<=4000. 

In [5]:
lmax_ivf, mmax_ivf, beam, nlev_t, nlev_p = (3000, 3000, 1., 1., np.sqrt(2.))

lmin_tlm, lmin_elm, lmin_blm = (30, 30, 30) # The fiducial transfer functions are set to zero below these lmins
# for delensing useful to cut much more B. It can also help since the cg inversion does not have to reconstruct those.

lmax_qlm, mmax_qlm = (4000, 4000) # Lensing map is reconstructed down to this lmax and mmax
# NB: the QEs from plancklens does not support mmax != lmax, but the MAP pipeline does
lmax_unl, mmax_unl = (4000, 4000) # Delensed CMB is reconstructed down to this lmax and mmax

#----------------- pixelization and geometry info for the input maps and the MAP pipeline and for lensing operations
nside = 2048
zbounds     = (-1.,1.) # colatitude sky cuts for noise variance maps (We could exclude all rings which are completely masked)

zbounds_len = (-1.,1.) # Outside of these bounds the reconstructed maps are assumed to be zero
pb_ctr, pb_extent = (0., 2 * np.pi) # Longitude cuts, if any, in the form (center of patch, patch extent)

lensres = 1.7  # Deflection operations will be performed at this resolution
Lmin = 2 # The reconstruction of all lensing multipoles below that will not be attempted
stepper = steps.nrstep(lmax_qlm, mmax_qlm, val=0.5) # handler of the size steps in the MAP BFGS iterative search

## geometry
 * When working with CMB data, most likely you will be working with healpix maps, which are defined by the healpix geometry.
 * There could be use cases in which you could want the geometry to be different.
 * D.lensalot uses Scarf, a python wrapper about DUCC to calculate the spherical harmonics.
 * DUCC itself supports various predefined, and custom geometries.
 * For more info, see https://scarfcmb.readthedocs.io/en/latest/geometry.html

In [ ]:
ninvjob_geometry = utils_scarf.Geom.get_healpix_geometry(nside, zbounds=zbounds)
lenjob_geometry = utils_scarf.Geom.get_thingauss_geometry(lmax_unl, 2, zbounds=zbounds_len)
lenjob_pbgeometry =utils_scarf.pbdGeometry(lenjob_geometry, utils_scarf.pbounds(pb_ctr, pb_extent))